In [1]:
import os
import pandas as pd
import logging
import io
import requests
import threading
import queue
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
from lxml import html

In [4]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_inn_vie_purch(purch,q):   
    url="https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber="+purch 
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table',{'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    else:
        q.put(0)
        return
    contract_link = soup.find('a')
    try:
        contract_url = contract_link.get('href')
    except Exception:
        q.put(0)
        return
    contract_url = 'https://zakupki.gov.ru' + contract_url
    try:
        contract_page = requests.get(contract_url, headers = headers)
        print(contract_url)
    except Exception:
        q.put(0)
        return
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span)+1
            break
    if ind_cur == -1:
        q.put(0)
        return 
    else:
        q.put(spans[ind_cur].text)
        return 
tqdm.pandas()

In [3]:
def start_download(purch):
    out = purch
    q = queue.Queue()
    scraping = threading.Thread(target=get_inn_vie_purch, args=(out,q,))
    scraping.start()
    scraping.join()
    while threading.active_count()>15:
        time.sleep(0.01)
    return q.get()

In [6]:
inn_list = ['7726311464',
'7724922443',
'4003032047',
'7707049388',
'7726416643',
'7718538045',
'7719555477',
'1101107584',
'0276065541',
'7716644263',
'7711027436',
'7736035485',
'6166083490',
'7810687137',
'2724070180',
'2723071046',
'7743529527',
'7707083893',
'5027262517',
'5012074091',
'2801244676',
'2540203506',
'7715656026',
'4719016624',
'6166084084',
'1901120601',
'3444197347',
'2801056707',
'7701213835',
'6319037334',
'5017112368',
'7733271117',
'7724419198',
'7718599175',
'7713056834',
'7733507267',
'7714511476',
'3459070488',
'5007103318',
'2312178145']

In [9]:
df = pd.read_csv('valid.csv')

In [22]:
df[df['winer'] == inn_list[0]].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [24]:
df.shape

(2227, 29)